In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import skfda
from skfda.datasets import fetch_growth
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import (
    BSplineBasis,
    FourierBasis,
    MonomialBasis,
)
import scipy 

from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score


In [10]:
def CFPCA(foreground,background, alpha, t, centered):
    X = np.array(foreground)
    Y = np.array(background)
    
    if not centered:
        X = X - np.mean(X, axis = 0)
        Y = Y - np.mean(Y, axis = 0)
    
    w = (t[-1]-t[0])/len(t)

    Vx = (1/(len(X)-1)) * np.dot(X.T, X)
    Vy = (1/(len(Y)-1)) * np.dot(Y.T, Y)

    # Perform the eigen decomposition on the constrastive covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(w*(Vx-alpha*Vy))

    # Sort the eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    return sorted_eigenvectors


In [5]:
def CFPCA_2(foreground, background, alpha, interval):
    fd_X = skfda.FDataGrid(foreground, interval)
    fd_Y = skfda.FDataGrid(background, interval)
    basis = skfda.representation.basis.BSplineBasis(n_basis=9)
    X_basis = fd_X.to_basis(basis)
    Y_basis = fd_Y.to_basis(basis)

    X_fd_data = fd_X.data_matrix.reshape(fd_X.data_matrix.shape[:-1])
    Y_fd_data = fd_Y.data_matrix.reshape(fd_Y.data_matrix.shape[:-1])

    X_identity = np.eye(len(fd_X.grid_points[0]))
    Y_identity = np.eye(len(fd_Y.grid_points[0]))

    X_weights = scipy.integrate.simpson(X_identity, fd_X.grid_points[0])
    Y_weights = scipy.integrate.simpson(Y_identity, fd_Y.grid_points[0])

    X_weights_matrix = np.diag(X_weights)
    Y_weights_matrix = np.diag(Y_weights)

    X_factorization_matrix = X_weights_matrix.astype(float)
    Y_factorization_matrix = Y_weights_matrix.astype(float)

    X_Lt = np.linalg.cholesky(X_factorization_matrix).T
    Y_Lt = np.linalg.cholesky(Y_factorization_matrix).T

    new_data_matrix_X = X_fd_data @ X_weights_matrix
    new_data_matrix_X = np.linalg.solve(X_Lt.T, new_data_matrix_X.T).T

    new_data_matrix_Y = Y_fd_data @ Y_weights_matrix
    new_data_matrix_Y = np.linalg.solve(Y_Lt.T, new_data_matrix_Y.T).T

    X_centered = new_data_matrix_X - np.mean(new_data_matrix_X, axis=0)
    Y_centered = new_data_matrix_Y - np.mean(new_data_matrix_Y, axis=0)

    Vx = (1 / (len(foreground) - 1)) * np.dot(X_centered.T, X_centered)
    Vy = (1 / (len(background) - 1)) * np.dot(Y_centered.T, Y_centered)

    # Perform the eigen decomposition on the covariance matrix V
    eigenvalues, eigenvectors = np.linalg.eig(Vx - alpha * Vy)

    # Sort the eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    components = np.linalg.solve(X_Lt, sorted_eigenvectors[:, :K]).T

    return components